In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import glob
import xarray as xr
from collections import defaultdict
from scipy.stats import gaussian_kde
from scipy.integrate import simpson
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import matplotlib as mpl
import pltkit
import read_files as rf
import generate_daily_temp as gdt
# from paf_calculations import get_daily_rr

wdir = 'X:\\user\\liprandicn\\Health Impacts Model'

### Plots RR

In [2]:
dfs = {}
file_path = f'{wdir}\\output'

for ccategory in pltkit.ccategories:
    for std in pltkit.std_values:

        method = 'm2'
        key = f"{ccategory}_{method}_{std}"
        file_name = f"rr_{method}_resp_copd_{ccategory}_2010-2100_{std}std.csv"
        Path = os.path.isfile(f"{file_path}\\{file_name}")
        if Path == True:
                dfs[key] = pd.read_csv(f"{file_path}\\{file_name}", index_col=[0])
                dfs[key] = dfs[key].set_index(pltkit.image_regions['Region']) 
  
            
        method = 'm1'
        key = f"{ccategory}_{method}_{std}"
        file_name = f"rr_{method}_all-diseases_{ccategory}_2010-2100_{std}std.csv"
        Path = os.path.isfile(f"{file_path}\\{file_name}")
        if Path == True:
            dfs[key] = pd.read_csv(f"{file_path}\\{file_name}", index_col=[0], header=[0,1])
            dfs[key].set_index(pltkit.image_regions['Region'], inplace=True)
            
            
        method = 'm1-1'
        key = f"{ccategory}_{method}_{std}"
        file_name = f"rr_{method}_all-diseases_{ccategory}_2010-2100_{std}std.csv"
        Path = os.path.isfile(f"{file_path}\\{file_name}")
        if Path == True:
            dfs[key] = pd.read_csv(f"{file_path}\\{file_name}", index_col=[0], header=[0,1,2])
            dfs[key].set_index(pltkit.image_regions['Region'], inplace=True)

#### Comparisson per IMAGE regions for one C-Category and disease

In [ ]:
ccategory = 'C8'
std = 5
disease = 'lri'
regions = dfs[f"{ccategory}_m1_{std}"].index 
colors = [pltkit.combined_cmap(i / len(regions)) for i in range(len(regions))]  

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 7))

for i, region in enumerate(regions):
    color = colors[i] 
    # ax1.plot(dfs[f'{ccategory}_m2_{std}'].columns, dfs[f'{ccategory}_m1_{std}'].xs(disease, axis=1, level=1).loc[region, :], label=region, color=color)
    # ax2.plot(dfs[f'{ccategory}_m2_{std}'].columns, dfs[f'{ccategory}_m2_{std}'].loc[region, :], label=region, color=color)
    # ax1.plot(dfs[f'{ccategory}_m2_{std}'].columns, dfs[f'{ccategory}_m1-1_{std}'].xs(disease, axis=1, level=1).xs('all', axis=1, level=1).loc[region, :], label=region, color=color)
    ax1.plot(dfs[f'{ccategory}_m2_{std}'].columns, dfs[f'{ccategory}_m1-1_{std}'].xs(disease, axis=1, level=1).xs('cold', axis=1, level=1).loc[region, :], label=region, color=color)
    ax2.plot(dfs[f'{ccategory}_m2_{std}'].columns, dfs[f'{ccategory}_m1-1_{std}'].xs(disease, axis=1, level=1).xs('hot', axis=1, level=1).loc[region, :], label=region, color=color)

common_kwargs = {'xlabel': 'Year', 'ylabel': 'RR', 'xticks': pltkit.ticks_years, 'xtickslabels': pltkit.ticks_labels_years}
pltkit.stylize_axes(ax1, title='M1-1 COLD', **common_kwargs)
pltkit.stylize_axes(ax2, title='M1-1 HOT', **common_kwargs)
# pltkit.stylize_axes(ax3, title='M1-1', **common_kwargs)

pltkit.stylize_plot(legend=True, legend_kwargs={'loc':'upper center', 'bbox_to_anchor':(-0.15, -0.1), 'ncol': 5},
                    suptitle=f'{ccategory} - {pltkit.diseases[disease]}', suptitle_kwargs={'fontsize': 14})

#### Comparisson per category for one region

In [ ]:
region = 'Russia region'
std = 5

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))
for ccategory in pltkit.ccategories:
    ax1.plot(dfs[f"{ccategory}_m1_{std}"].xs(disease, axis=1, level=1).columns, dfs[f"{ccategory}_m1_{std}"].xs(disease, axis=1, level=1).loc[region, :], label=ccategory)
    ax2.plot(dfs[f"{ccategory}_m2_{std}"].columns, dfs[f"{ccategory}_m2_{std}"].loc[region,:], label=ccategory)

common_kwargs = {'xlabel': 'Year', 'ylabel': 'RR', 'xticks': pltkit.ticks_years, 'xtickslabels': pltkit.ticks_labels_years}
pltkit.stylize_axes(ax1, title='M1', **common_kwargs)
pltkit.stylize_axes(ax2, title='M2', **common_kwargs)

pltkit.stylize_plot(suptitle=region, suptitle_kwargs={'fontsize': 14}, legend=True)

#### Removing POP

In [ ]:
file_name = f"rr_{method}_all-diseases_C1_2010-2100_{std}std_POP.csv"
df_m1 = pd.read_csv(f"{file_path}\\{file_name}", index_col=[0], header=[0,1])
df_m1.set_index(pltkit.image_regions['Region'], inplace=True)

ccategory = 'C1'
std = 5
disease = 'resp_copd'
regions = dfs[f"{ccategory}_m1_{std}"].index 
colors = [pltkit.combined_cmap(i / len(regions)) for i in range(len(regions))]  

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 7))

for i, region in enumerate(regions):
    color = colors[i] 
    ax1.plot(dfs[f'{ccategory}_m1_{std}'].xs(disease, axis=1, level=1).columns, dfs[f'{ccategory}_m1_{std}'].xs(disease, axis=1, level=1).loc[region, :], label=region, color=color)
    ax2.plot(df_m1.xs(disease, axis=1, level=1).columns, df_m1.xs(disease, axis=1, level=1).loc[region, :], label=region, color=color)

common_kwargs = {'xlabel': 'Year', 'ylabel': 'RR', 'xticks': pltkit.ticks_years, 'xtickslabels': pltkit.ticks_labels_years}
pltkit.stylize_axes(ax1, title='M1 - SSP2 POP', **common_kwargs)
pltkit.stylize_axes(ax2, title='M2 - 2010 POP', **common_kwargs)

pltkit.stylize_plot(legend=True, legend_kwargs={'loc':'upper center', 'bbox_to_anchor':(-0.15, -0.1), 'ncol': 5},
    suptitle=f'{ccategory} - {disease}', suptitle_kwargs={'fontsize': 14}, show=True)

### Role of STD

#### Plots without noise

In [ ]:
ccategory =  'C8'
std_values = '5'
disease = 'resp_copd'

df_wonoise_m1 = pd.read_csv(f"{wdir}\\output\\rr_m1_all-diseases_{ccategory}_2010-2100_{std_values}std_wo-noise.csv", index_col=[0], header=[0,1])
df_wonoise_m1.set_index(pltkit.image_regions['Region'], inplace=True)
df_wonoise_m2 = pd.read_csv(f"{wdir}\\output\\rr_m2_resp_copd_{ccategory}_2010-2100_{std_values}std_wo-noise.csv", index_col=[0])
df_wonoise_m2.set_index(pltkit.image_regions['Region'], inplace=True)

regions = df_wonoise_m2.index 
colors = [pltkit.combined_cmap(i / len(regions)) for i in range(len(regions))]  

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 7))

for i, region in enumerate(regions):
    color = colors[i] 
    ax1.plot(df_wonoise_m1.xs(disease, axis=1, level=1).columns, df_wonoise_m1.xs(disease, axis=1, level=1).loc[region, :], label=region, color=color)
    ax2.plot(df_wonoise_m2.columns, df_wonoise_m2.loc[region, :], label=region, color=color)

common_kwargs = {'xlabel': 'Year', 'ylabel': 'RR', 'xticks': pltkit.ticks_years, 'xtickslabels': pltkit.ticks_labels_years}
pltkit.stylize_axes(ax1, title='M1', **common_kwargs)
pltkit.stylize_axes(ax2, title='M2', ylim=(1,1.18), **common_kwargs)
pltkit.stylize_plot(legend=True, legend_kwargs={'loc':'upper center', 'bbox_to_anchor':(-0.15, -0.1), 'ncol': 5},
    suptitle=f'{ccategory} - {disease}', suptitle_kwargs={'fontsize': 14}, show=True)

In [ ]:
region = 'Western Europe'
std = 5

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))
ax1.plot(df_wonoise_m1.xs(disease, axis=1, level=1).columns, df_wonoise_m1.xs(disease, axis=1, level=1).loc[region, :], label=ccategory)
ax2.plot(df_wonoise_m2.columns, df_wonoise_m2.loc[region,:], label=ccategory)

tick_positions = [0, 20, 40, 60, 80]  # Posiciones en el eje x
tick_labels = dfs[f"{ccategory}_m2_{std}"].columns.values[tick_positions]  # Etiquetas correspondientes

common_kwargs = {'xlabel': 'Year', 'ylabel': 'RR', 'xticks': tick_positions, 'xtickslabels': tick_labels}
pltkit.stylize_axes(ax1, title='M1', **common_kwargs)
pltkit.stylize_axes(ax2, title='M2', **common_kwargs)

pltkit.stylize_plot(legend=True, suptitle=f'{region}', suptitle_kwargs={'fontsize': 14}, show=True)

In [8]:
pdf_wonoise = pd.read_csv(f"{wdir}\\output\\dailyTemp_C8_2010-2100_{std}std_wo-noise.csv", index_col=[0,1])
pdf_wonoise.index = pd.MultiIndex.from_product([pltkit.image_regions['Region'], pdf_wonoise.index.get_level_values(1)[:1000]])

In [ ]:
region = 'Oceania'

pdf_smooths = {}
temps = pdf_wonoise.loc[region].index.to_numpy()
for year in ['2010', '2030', '2050', '2100']:
    pdf_values = pdf_wonoise.loc[region][year].fillna(0).to_numpy()
    pdf_values /= pdf_values.sum() 
    kde = gaussian_kde(temps, weights=pdf_values)
    temps_smooth = np.linspace(temps.min(), temps.max(), 500)  
    pdf_smooths[year] = kde(temps_smooth)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(temps_smooth, pdf_smooths['2010'], color='C0', label='2010')
ax1.axvline(x= (pdf_wonoise.loc[region].index*pdf_wonoise.loc[region]['2010']).sum(), color ='C0', ls='--')
ax1.plot(temps_smooth, pdf_smooths['2050'], color='C1', label='2050')
ax1.axvline(x= (pdf_wonoise.loc[region].index*pdf_wonoise.loc[region]['2050']).sum(), color ='C1', ls='--')
ax1.plot(temps_smooth, pdf_smooths['2100'], color='C2', label='2100')
ax1.axvline(x= (pdf_wonoise.loc[region].index*pdf_wonoise.loc[region]['2100']).sum(), color ='C2', ls='--')

ax2.plot(df_wonoise_m1.xs(disease, axis=1, level=1).columns, df_wonoise_m1.xs(disease, axis=1, level=1).loc[region], label='M1', color='C5')
ax2.plot(df_wonoise_m2.columns, df_wonoise_m2.loc[region, :], label='M2', color='C4')

pltkit.stylize_axes(ax1, title=f'PDF 2010 vs 2100', xlabel="Daily temperature (°C)", ylabel="Probability Density",
             legend=True, grid=True)
pltkit.stylize_axes(ax2, title=f'M1 vs M2 - {disease}', xlabel="Year", ylabel="RR", xticks=tick_positions, xtickslabels=tick_labels,
             legend=True, grid=True)
pltkit.stylize_plot(suptitle=f'{region} - {ccategory} - Without noise', suptitle_kwargs={'fontsize': 14}, show=True)

#### Comparisson per STD for all regions

In [ ]:
ccategories_ = ['C1', 'C8']
std_values = [1,5,10]
disease = 'resp_copd'
regions = dfs[f"{ccategory}_m1_{std}"].index 
colors = [pltkit.combined_cmap(i / len(regions)) for i in range(len(regions))]  

fig, axs = plt.subplots(2, 3, figsize=(15, 10))

for i, ccategory in enumerate(ccategories_):
    for j, std in enumerate(std_values):
        for k, region in enumerate(regions[:-1]):
            color = colors[k] 
            # axs[i,j].plot(dfs[f'{ccategory}_m1_{std}'].xs(disease, axis=1, level=1).columns, dfs[f'{ccategory}_m1_{std}'].xs(disease, axis=1, level=1).loc[region, :], label=region, color=color)
            axs[i,j].plot(dfs[f'{ccategory}_m2_{std}'].columns, dfs[f'{ccategory}_m2_{std}'].loc[region, :], label=region, color=color)

for ax in axs.flat:
    pltkit.stylize_axes(ax, xticks=pltkit.ticks_years, xtickslabels=pltkit.ticks_labels_years, ylim=(1.0, 1.19))  
    
for i, ax in enumerate(axs[0,:]):
    ax.set_title(f'std_{std_values[i]}')
    
axs[0,0].set_ylabel('RR - C1')
axs[1,0].set_ylabel('RR - C8')
plt.legend(loc='upper center', bbox_to_anchor=(-0.7, -0.1), ncol=6) 

#### Comparisson per STD per IMAGE region

In [ ]:
region = 'South Africa'

fig, axs = plt.subplots(2, 3, figsize=(16, 8))

for i, method in enumerate(pltkit.methods):
    for j, std in enumerate(pltkit.std_values):
        for k, ccategory in enumerate(pltkit.ccategories):
            if i == 0 and f"{ccategory}_{method}_{std}" in dfs.keys():
                axs[i,j].plot(dfs[f"{ccategory}_{method}_{std}"].xs(disease, axis=1, level=1).columns, dfs[f"{ccategory}_{method}_{std}"].xs(disease, axis=1, level=1).loc[region,:],
                              label=ccategory, color=pltkit.original_colors[k])
            if i == 1 and f"{ccategory}_{method}_{std}" in dfs.keys():
                axs[i,j].plot(dfs[f"{ccategory}_{method}_{std}"].columns, dfs[f"{ccategory}_{method}_{std}"].loc[region,:], label=ccategory, color=pltkit.original_colors[k])
        
        pltkit.stylize_axes(axs[i,j], xticks=pltkit.ticks_years, xtickslabels=pltkit.ticks_labels_years, xlabel='Year', grid=True)    
        axs[0,j].set_title(f'{std}')    
        axs[i,0].set_ylabel(f'RR - {method}')
        axs[0,j].set_ylim(1.015,1.11)
        axs[1,j].set_ylim(1.05,1.13)
pltkit.stylize_plot(legend=True, suptitle=region, suptitle_kwargs={'fontsize':14})        

### Plot PDFs

In [12]:
df_pdfs = {}
for ccategory in pltkit.ccategories:
    for std in [1,5,10]:# std_values:
        key = f"{ccategory}_{std}"
        df_pdfs[key] = pd.read_csv(f"{wdir}\\output\\dailyTemp_{ccategory}_2010-2100_{std}std.csv", index_col=[0,1])
        df_pdfs[key].index = pd.MultiIndex.from_product([pltkit.image_regions['Region'], df_pdfs[key].index.get_level_values(1)[:1000]])
        
    for year in [1990, 2010, 2019]:
        key = f'era5_{year}'
        df_pdfs[key] = pd.read_csv(f"{wdir}\\output\\dailyTemp_{year}_era5.csv", index_col=[0,1])
        df_pdfs[key].index = pd.MultiIndex.from_product([pltkit.image_regions['Region'], df_pdfs[key].index.get_level_values(1)[:1000]])

#### Normal PDF (STD = 5)

In [ ]:
region = 'Canada'
ccategory = 'C1'
std = 5

pdf_smooths = {}
temps = df_pdfs[f'{ccategory}_{std}'].loc[region].index.to_numpy()

for year in ['2010', '2050', '2100']:
    pdf_values = df_pdfs[f'{ccategory}_{std}'].loc[region][year].fillna(0).to_numpy()
    pdf_values /= pdf_values.sum() 
    kde = gaussian_kde(temps, weights=pdf_values)
    temps_smooth = np.linspace(temps.min(), temps.max(), 1000)  
    pdf_smooths[year] = kde(temps_smooth)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(temps_smooth, pdf_smooths['2010'], color='C0', label='2010')
ax1.axvline(x= (df_pdfs[f'{ccategory}_{std}'].loc[region].index*df_pdfs[f'{ccategory}_{std}'].loc[region]['2010']).sum(), color ='C0', ls='--')
ax1.plot(temps_smooth, pdf_smooths['2050'], color='C1', label='2050')
ax1.axvline(x= (df_pdfs[f'{ccategory}_{std}'].loc[region].index*df_pdfs[f'{ccategory}_{std}'].loc[region]['2050']).sum(), color ='C1', ls='--')
ax1.plot(temps_smooth, pdf_smooths['2100'], color='C2', label='2100')
ax1.axvline(x= (df_pdfs[f'{ccategory}_{std}'].loc[region].index*df_pdfs[f'{ccategory}_{std}'].loc[region]['2100']).sum(), color ='C2', ls='--')

ax2.plot(dfs[f'{ccategory}_m1_{std}'].xs(disease, axis=1, level=1).columns, dfs[f'{ccategory}_m1_{std}'].xs(disease, axis=1, level=1).loc[region], label='M1', color='C5')
ax2.plot(dfs[f'{ccategory}_m2_{std}'].columns, dfs[f'{ccategory}_m2_{std}'].loc[region, :], label='M2', color='C4')

pltkit.stylize_axes(ax1, title=f'PDF 2010 vs 2100', xlabel="Daily temperature (°C)", ylabel="Probability Density",
             legend=True, grid=True)
pltkit.stylize_axes(ax2, title=f'M1 vs M2 - {disease}', xlabel="Year", ylabel="RR", xticks=tick_positions, xtickslabels=tick_labels,
             legend=True, grid=True)
pltkit.stylize_plot(suptitle=f'{region} - {ccategory}', suptitle_kwargs={'fontsize': 14}, show=True)

#### Experiment PDFs x RRs

In [ ]:
erf = pd.read_csv(f'{wdir}\\Response_Functions\\erf_reformatted2\\erf_resp_copd.csv', index_col=[1,2])
tz = 6
erf_tz = erf.loc[tz]
erf_tz = erf_tz.reindex(np.arange(-40,60,0.1).round(1))
erf_tz = erf_tz.ffill().bfill()
for year in [2010, 2050, 2100]:
    erf_tz[year] = pdf_smooths[f'{year}'] * erf_tz['rr']
    
plt.plot(erf_tz.index, erf_tz[2010], label='2010')
plt.plot(erf_tz.index, erf_tz[2050], label='2050')
plt.plot(erf_tz.index, erf_tz[2100], label='2100')
plt.axvline(x=erf_tz.index[600], color='k', linestyle='--')

print("area_2010_hot =", simpson(erf_tz[2010].iloc[600:], dx=0.1))
print('area_2010_cold =', simpson(erf_tz[2010].iloc[:600], dx=0.1))

print("area_2050_hot =", simpson(erf_tz[2050].iloc[600:], dx=0.1))
print('area_2050_cold =', simpson(erf_tz[2050].iloc[:600], dx=0.1))

print("area_2100_hot =", simpson(erf_tz[2100].iloc[600:], dx=0.1))
print('area_2100_cold =', simpson(erf_tz[2100].iloc[:600], dx=0.1))

#### Plots to study the role of STD

In [ ]:
# region = 'Korea region'
ccategory = 'C1'

pdf_smooths = {}
temps = df_pdfs[f'{ccategory}_{std}'].loc[region].index.to_numpy()
    
fig, axs = plt.subplots(1, 3, figsize=(16, 4))

for i,std in enumerate([1,5,10]):
    for year in ['2010', '2050', '2100']:
        pdf_values = df_pdfs[f'{ccategory}_{std}'].loc[region][year].fillna(0).to_numpy()
        pdf_values /= pdf_values.sum() 
        kde = gaussian_kde(temps, weights=pdf_values)
        temps_smooth = np.linspace(temps.min(), temps.max(), 500)  
        pdf_smooths[f'{std}_{year}'] = kde(temps_smooth)
        
        axs[i].plot(temps_smooth, pdf_smooths[f'{std}_{year}'], label=year)
        #axs[i].axvline(x= (df_pdfs[f'{ccategory}_{std}'].loc[region].index*df_pdfs[f'{ccategory}_{std}'].loc[region][year]).sum(), ls='--')
        
        pltkit.stylize_axes(axs[i], xlim=(0,60), ylim=(0,0.07), title=f'std_{std}', xlabel="Daily temperature (°C)",
                     legend=True, grid=True)
axs[0].set_ylabel("Probability Density") 
print(region)

#### ERA5 vs dummy data PDFs

In [ ]:
region = 'Canada'
ccategory = 'C1'
std = 5

pdf_smooths = {}
pdf_smooths_era5 = {}
temps = df_pdfs[f'{ccategory}_{std}'].loc[region].index.to_numpy()


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

for year in ['2010', '2019']:
    pdf_values = df_pdfs[f'{ccategory}_{std}'].loc[region][year].fillna(0).to_numpy()
    pdf_values /= pdf_values.sum() 
    kde = gaussian_kde(temps, weights=pdf_values)
    temps_smooth = np.linspace(temps.min(), temps.max(), 1000)  
    pdf_smooths[year] = kde(temps_smooth)

    ax1.plot(temps_smooth, pdf_smooths[year], label=year)
    ax1.axvline(x= (df_pdfs[f'{ccategory}_{std}'].loc[region].index*df_pdfs[f'{ccategory}_{std}'].loc[region][year]).sum(), ls='--')
    
    pdf_values_era5 = df_pdfs[f'era5_{year}'].loc[region][year].fillna(0).to_numpy()
    pdf_values_era5 /= pdf_values_era5.sum() 
    kde_era5 = gaussian_kde(temps, weights=pdf_values_era5)
    temps_smooth_era5 = np.linspace(temps.min(), temps.max(), 1000)  
    pdf_smooths_era5[year] = kde_era5(temps_smooth_era5)

    ax2.plot(temps_smooth_era5, pdf_smooths_era5[year], label=year)
    ax2.axvline(x= (df_pdfs[f'era5_{year}'].loc[region].index*df_pdfs[f'era5_{year}'].loc[region][year]).sum(), ls='--')

for ax in (ax1, ax2):
    # ax.set_xlim(0,40)
    ax.legend()


### Plots Present Day RR

In [ ]:
###Select year
year = 2019

### Select specific year and "all-ages" group for GBD mortality data
gbd_mortality = pltkit.gbd_mortality[(pltkit.gbd_mortality['year'] == year) & (pltkit.gbd_mortality['age'] == 'All ages')]

# Make dictionary with mapping and invert it for mapping
image_regions_dic = pltkit.image_regions.set_index('Region')['GBD region'].apply(lambda x: x.split(', ')).to_dict()
location_to_image_region = {loc: key for key, locations in image_regions_dic.items() for loc in locations}

# Sum deaths per IMAGE region
gbd_mortality["image_region"] = gbd_mortality["location"].map(location_to_image_region)
gbd_mortality_image_region = gbd_mortality.groupby(["image_region", "cause"], as_index=False).agg({"val": "sum"})#, "upper": "sum", "lower": "sum"})
gbd_mortality_image_region = gbd_mortality_image_region.pivot(index="image_region", columns="cause", values="val").fillna(0).rename_axis(None, axis=1)

In [ ]:
gbd_mortality_image_region.sum(axis=0)

In [ ]:
gbd_mortality.cause.unique()

In [ ]:
### Load RRs dataframes and calculate the total attributable burden per disease and IMAGE region
rr = {}
for method in ['m1', 'm2']:
    rr[f'rr_{method}'] = pd.read_csv(f'{wdir}\\output\\rr_{method}_all-diseases_{year}_era5.csv', index_col=[0], header=[0,1])
    rr[f'rr_{method}'].set_index(pltkit.image_regions['Region'], inplace=True)
    rr[f'rr_{method}'] = rr[f'rr_{method}'].xs(f'{year}', axis=1, level=0)
    rr[f'paf_{method}'] = pd.DataFrame().reindex_like(rr[f'rr_{method}'])
    for disease in pltkit.diseases:
        rr[f'paf_{method}'][disease] = 1 - (1/rr[f'rr_{method}'][disease])
    rr[f'mortality_{method}'] = pd.DataFrame().reindex_like(gbd_mortality_image_region)
    for disease in pltkit.diseases:
        rr[f'mortality_{method}'][pltkit.diseases[disease]] = rr[f'paf_{method}'][disease] * gbd_mortality_image_region[pltkit.diseases[disease]] 
        
print(f'Global Attributable Burden: M1 = {int(rr[f'mortality_m1'].sum(axis=0).sum())}; M2 = {int(rr[f'mortality_m2'].sum(axis=0).sum())}')

In [ ]:
dfs[f'rr_m2'] = pd.read_csv(f"{wdir}\\output\\rr_m2_all-diseases_C1_2010-2019_5std.csv", index_col=[0], header=[0,1])
dfs[f'rr_m2'].set_index(pltkit.image_regions['Region'], inplace=True)
dfs[f'rr_m2'] = dfs[f'rr_m2'].xs(f'{year}', axis=1, level=0)

dfs[f'rr_m1'] = dfs[f'C1_m1_5'].xs(f'{year}', axis=1, level=0)

for method in ['m1', 'm2']:
        dfs[f'paf_{method}'] = pd.DataFrame().reindex_like(rr[f'rr_{method}'])
        for disease in pltkit.diseases:
                dfs[f'paf_{method}'][disease] = 1 - (1/dfs[f'rr_{method}'][disease])
        dfs[f'mortality_{method}'] = pd.DataFrame().reindex_like(gbd_mortality_image_region)
        for disease in pltkit.diseases:
                dfs[f'mortality_{method}'][pltkit.diseases[disease]] = dfs[f'paf_{method}'][disease] * gbd_mortality_image_region[pltkit.diseases[disease]] 

print(f'Global Attributable Burden: M1 = {int(dfs[f'mortality_m1'].sum(axis=0).sum())}; M2 = {int(dfs[f'mortality_m2'].sum(axis=0).sum())}')

In [ ]:
method = 'm1'
cumval = 0
fig = plt.figure(figsize=(12, 8))

# Generate a color map for the diseases
num_colors = len(pltkit.diseases.keys())
cmap = cm.get_cmap('tab20', num_colors)  
disease_colors = {disease: cmap(i) for i, disease in enumerate(pltkit.diseases.values())}

df = dfs[f'mortality_{method}'].drop(['Environmental heat and cold exposure'], axis=1)
# Generate the stacked bar plot
for i, col in enumerate(df.columns):
    color = disease_colors.get(col)  
    plt.bar(df.index, df[col], bottom=cumval, color=color, label=col)
    cumval = cumval + df[col]

pltkit.stylize_plot(ylabel='Total number of deaths', title=f'Total attributable deaths in {year} - {method}', 
             legend=True, xticks_kwargs={'rotation': 90}, show=True)

In [ ]:
cumval = 0
fig = plt.figure(figsize=(12, 8))

# Generar el gráfico de barras con colores dinámicos
for i, col in enumerate(pltkit.relevant_diseases.keys()):
    color = disease_colors.get(pltkit.relevant_diseases[col], 'gray') 
    plt.bar(df.index, df[pltkit.relevant_diseases[col]], bottom=cumval, color=color, label=pltkit.relevant_diseases[col])
    cumval = cumval + df[pltkit.relevant_diseases[col]]

pltkit.stylize_plot(ylabel='Total number of deaths', legend=True, xticks_kwargs={'rotation': 90}, show=True)

#### Plots from python

In [ ]:
year = 2019
image_regions = pd.read_csv(f'X:\\user\\liprandicn\\Health Impacts Model\\data\\IMAGE_regions\\IMAGE_regions.csv',  index_col=0, header=0)

### Load GBD mortality data and select specific year and "all-ages" group
gbd_mortality = pd.read_csv(f'X:\\user\\liprandicn\\Health Impacts Model\\data\\GBD_Data\\Mortality\\IHME-GBD_2021_DATA-c46fcd58.csv')
gbd_mortality = gbd_mortality[(gbd_mortality['year'] == year) & (gbd_mortality['age'] == 'All ages')]

# Make dictionary with mapping and invert it for mapping
image_regions_dic = image_regions.set_index('Region')['GBD region'].apply(lambda x: x.split(', ')).to_dict()
location_to_image_region = {loc: key for key, locations in image_regions_dic.items() for loc in locations}

# Sum deaths per IMAGE region
gbd_mortality["image_region"] = gbd_mortality["location"].map(location_to_image_region)
gbd_mortality_image_region = gbd_mortality.groupby(["image_region", "cause"], as_index=False).agg({"val": "sum"})#, "upper": "sum", "lower": "sum"})
gbd_mortality_image_region = gbd_mortality_image_region.pivot(index="image_region", columns="cause", values="val").fillna(0).rename_axis(None, axis=1)

rr_year = pd.read_csv(f'{wdir}\\output\\rr_m1-1_all-diseases_{year}_era5.csv', header=[0,1,2])
rr_year.set_index(image_regions['Region'], inplace=True)

In [ ]:
diseases_full_names = {'ckd':'Chronic kidney disease', 'cvd_cmp':'Cardiomyopathy and myocarditis', 'cvd_htn':'Hypertensive heart disease', 
            'cvd_ihd':'Ischemic heart disease', 'cvd_stroke':'Stroke', 'diabetes':'Diabetes mellitus',
            'inj_animal':'Animal contact', 'inj_disaster':'Exposure to forces of nature', 'inj_drowning':'Drowning', 
            'inj_homicide':'Interpersonal violence', 'inj_mech':'Exposure to mechanical forces', 
            'inj_othunintent':'Other unintentional injuries', 'inj_suicide':'Self-harm', 'inj_trans_other':'Other transport injuries', 
            'inj_trans_road':'Road injuries', 'resp_copd':'Chronic obstructive pulmonary disease', 'lri':'Lower respiratory infections'}

In [ ]:
dfs = {}
dfs['rr_all'] = rr_year.xs('all', axis=1, level=2).xs(f'{year}', axis=1, level=0)
dfs[f'rr_cold'] = rr_year.xs('cold', axis=1, level=2).xs(f'{year}', axis=1, level=0)
dfs[f'rr_hot'] = rr_year.xs('hot', axis=1, level=2).xs(f'{year}', axis=1, level=0)


for method in ['all', 'cold', 'hot']:
        dfs[f'paf_{method}'] = pd.DataFrame().reindex_like(dfs[f'rr_{method}'])
        for disease in diseases_full_names:
                dfs[f'paf_{method}'][disease] = 1 - (1/dfs[f'rr_{method}'][disease])
        dfs[f'mortality_{method}'] = pd.DataFrame().reindex_like(gbd_mortality_image_region)
        for disease in diseases_full_names:
                dfs[f'mortality_{method}'][diseases_full_names[disease]] = dfs[f'paf_{method}'][disease] * gbd_mortality_image_region[diseases_full_names[disease]] 

print(f'Global Attributable Burden: ALL = {int(dfs[f'mortality_all'].sum(axis=0).sum())}; COLD = {int(dfs[f'mortality_cold'].sum(axis=0).sum())}; HOT = {int(dfs[f'mortality_hot'].sum(axis=0).sum())}')

In [ ]:
import matplotlib.cm as cm

method = 'hot'
cumval = 0
fig = plt.figure(figsize=(12, 8))

# Generate a color map for the diseases
num_colors = len(diseases_full_names.keys())
cmap = cm.get_cmap('tab20', num_colors)  
disease_colors = {disease: cmap(i) for i, disease in enumerate(diseases_full_names.values())}

df = dfs[f'mortality_{method}'][[diseases_full_names[d] for d in diseases_full_names.keys()]]
# Generate the stacked bar plot
for i, col in enumerate(df.columns):
    color = disease_colors.get(col)  
    plt.bar(df.index, df[col], bottom=cumval, color=color, label=col)
    cumval = cumval + df[col]

plt.xticks(rotation=90)
plt.ylabel('Total number of deaths')
plt.title(f'Total attributable deaths in {year} - {method} deaths')
plt.legend()
plt.show()

### IMAGE regions and Temperature Zones

In [ ]:
### Import ERA5 temperature zones and IMAGE region data
greg, era5_tz = read_IMAGEregions_and_TempZone()

### Import population data and create mask with population > 0 
ssp = 'SSP2_CP'
pop_ssp = get_annual_pop(f'{ssp}')
pop_ssp = pop_ssp.mean(dim='time').GPOP.values
valid_mask = pop_ssp > 0.

era5_tz = era5_tz[valid_mask]
greg = greg[valid_mask]

regions = pltkit.image_regions['Region']

### Assign color map
cmap = plt.get_cmap("tab20")  
colors = [cmap(i / len(regions)) for i in range(len(regions))]  

In [ ]:
fig, axes = plt.subplots(nrows=6, ncols=5, figsize=(20, 12))

# Graficar regiones 1 a 25
for region in range(1, 26):
    row = (region - 1) // 5
    col = (region - 1) % 5
    ax = axes[row, col]
    
    ax.grid(True, color='white', zorder=0)
    ax.set_facecolor('whitesmoke')
    ax.tick_params(colors='grey')  # ticks en gris
    for spine in ax.spines.values():
        spine.set_visible(False)  # quitar contornos

    color = colors[region - 1]
    valid_mask = greg == region
    era5_tz_region = era5_tz[valid_mask]
    era5_tz_region = era5_tz_region[~np.isnan(era5_tz_region)]

    unique_values = np.unique(era5_tz_region)
    ax.hist(era5_tz_region, bins=len(unique_values), density=True, alpha=0.5,
            label=image_regions.iloc[region - 1, 0], color=color, zorder=2)
    ax.set_title(f'{image_regions.iloc[region - 1, 0]}', color='dimgrey')


# Región 26 en el centro de la última fila
region = 26
ax = axes[5, 2]

ax.set_facecolor('whitesmoke')
ax.grid(True, color='white')
ax.tick_params(colors='grey')
for spine in ax.spines.values():
    spine.set_visible(False)

color = colors[region - 1]
valid_mask = greg == region
era5_tz_region = era5_tz[valid_mask]
era5_tz_region = era5_tz_region[~np.isnan(era5_tz_region)]

unique_values = np.unique(era5_tz_region)
ax.hist(era5_tz_region, bins=len(unique_values), density=True, alpha=0.5,
        label=image_regions.iloc[region - 1, 0], color=color, zorder=2)
ax.set_title(f'{image_regions.iloc[region - 1, 0]}', color='dimgrey')

# Eliminar los subplots vacíos en la última fila
for col in [0, 1, 3, 4]:
    fig.delaxes(axes[5, col])
    ax.hist(era5_tz_region, bins=len(unique_values), density=True, alpha=0.5, label=pltkit.image_regions.iloc[region-1, 0], color=color)
    
    #ax.plot(np.sort(era5_tz_region), pdf, color=color)
    ax.set_title(f'{pltkit.image_regions.iloc[region-1, 0]}')

plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111)
x_eval = np.linspace(6., 27., 1000)

for region in range(12,27):
    color = colors[region-1] 
    valid_mask = greg == region
    era5_tz_region = era5_tz[valid_mask]
    era5_tz_region = era5_tz_region[~np.isnan(era5_tz_region)]
    
    kde = gaussian_kde(era5_tz_region, bw_method='silverman')
    ax.plot(x_eval, kde(x_eval), label = pltkit.image_regions.iloc[region-1,0], color=color)
    
#plt.ylim(0,0.5)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=6, prop={'size': 8})

#### Temperature zones analysis

#### Temperature zones vs TMRELs

#### Population weighted PDFs

In [ ]:
pop_ssp = get_annual_pop('SSP2_CP')
pop_ssp_year = pop_ssp.sel(time=str(2019)).mean('time')
tmrel = {}
for year in [1990,2010,2020]:
    tmrel[year] = xr.open_dataset(f'{wdir}\\GBD_Data\\Exposure_Response_Functions\\TMRELs_{year}.nc')
era5_tz = xr.open_dataset(f'{wdir}\\Climate_Data\\ERA5\\ERA5_mean_1980-2019_land_t2m_tz.nc')

In [ ]:
counts_tmrel = defaultdict(dict); weights_tmrel = defaultdict(dict)

for year in [1990, 2010, 2020]:
    for tz in range(6,29):
        counts_tmrel[year][tz], weights_tmrel[year][tz] = pltkit.get_counts_and_weights_for_kde(tmrel[year], era5_tz, pop_ssp_year,tz)

In [ ]:
### Plot for one year
year = 1990
tz_values = list(range(6, 29))
colors = cm.copper(np.linspace(0, 1, len(tz_values)))

fig, ax = plt.subplots(figsize=(10, 8))#, dpi=300)
ax.set_facecolor('whitesmoke')

for i, tz in enumerate(tz_values):
    data = counts_tmrel[year][tz]
    w = weights_tmrel[year][tz]

    if len(data) > 50000:
        idx = np.random.choice(len(data), size=50000, replace=False, p=w)
        data = data[idx]
        w = w[idx]
        w = w / w.sum()

    kde = gaussian_kde(data, weights=w)

    p5, p95 = np.percentile(data, [5, 95])
    x = np.linspace(p5, p95, 200)
    y = kde(x)

    scale = 0.4 / y.max()
    y_scaled = y * scale

    ax.fill_betweenx(x, tz, tz + y_scaled, color=colors[i], alpha=0.8, zorder=2)

pltkit.stylize_axes(ax, xticks=tz_values, xtickslabels=tz_values, xlim=(5.5,28.5), ylim=(14,30), xlabel='Temperature zone [°C]',
             ylabel="TMREL [°C]", title=f"Population-weighted {year} TMREL Distributions by Temperature Zone\n5–95% Percentile Range",
             grid=True, grid_kwargs={'axis':'both', 'color':'white', 'zorder':0})

In [ ]:
### Plot for all years

tz_values = list(range(6, 29))
colors = cm.copper(np.linspace(0, 1, len(tz_values)))

fig, axs = plt.subplots(1,3, figsize=(18, 5))#, dpi=300)
axs=axs.flatten()

for j,year in enumerate([1990,2010,2020]):
    for i, tz in enumerate(tz_values):
        data = counts_tmrel[year][tz]
        w = weights_tmrel[year][tz]

        if len(data) > 50000:
            idx = np.random.choice(len(data), size=50000, replace=False, p=w)
            data = data[idx]
            w = w[idx]
            w = w / w.sum()

        kde = gaussian_kde(data, weights=w)

        p5, p95 = np.percentile(data, [5, 95])
        x = np.linspace(p5, p95, 200)
        y = kde(x)

        scale = 0.4 / y.max()
        y_scaled = y * scale

        axs[j].fill_betweenx(x, tz, tz + y_scaled, color=colors[i], alpha=0.9, zorder=2)

    pltkit.stylize_axes(axs[j], xticks=tz_values, xtickslabels=tz_values, xlim=(5.5,28.5), ylim=(13.5,30), xlabel='Temperature zone [°C]',
                ylabel="TMREL [°C]", title=f"{year}", grid=True, grid_kwargs={'axis':'both', 'color':'white', 'zorder':0}, facecolor='whitesmoke')
plt.suptitle('Population-weighted TMREL Distributions by Temperature Zone')

#### Temperature zones vs GDP

In [ ]:
# Open dataframes
gdp_data = pd.read_excel('X:\\user\\hooijschue\\Projects\\GDP_POP\\source_data update (elena)\\GDP\\iamc_db gdp.xlsx')
gdppc_data = pd.read_excel('X:\\user\\hooijschue\\Projects\\GDP_POP\\source_data update (elena)\\GDP\\iamc_db gdppc.xlsx')
# Open xarrays
gdp_original = xr.open_dataset(f'{wdir}\\SocioeconomicData\\EconomicData\\GDP_SSP2_2020_OriginalCountries.nc')
gdp = xr.open_dataset(f'{wdir}\\SocioeconomicData\\EconomicData\\GDP_SSP2_2020.nc')

#### Selected countries

In [ ]:
gdp_countries = gdp_data[(gdp_data['Region'].isin(pltkit.original_countries)) & (gdp_data['Scenario'] == 'SSP2')][['Region', 2020]].rename(columns={2020:'gdp'})
gdppc_countries = gdppc_data[(gdppc_data['Region'].isin(pltkit.original_countries)) & (gdppc_data['Scenario'] == 'SSP2')][['Region', 2020]].rename(columns={2020:'gdppc'})
    
counts_gdp, values_gdp, _ = get_values_counts_weights_for_bubbles(gdp_original.gdp, era5_tz.t2m, pop_ssp_year.GPOP, weighted=False)
counts_gdppc, values_gdppc, _ = get_values_counts_weights_for_bubbles(gdp_original.gdppc, era5_tz.t2m, pop_ssp_year.GPOP, weighted=False)

mapping = gdp_countries.set_index('gdp')['Region'].to_dict()
mapping_gdppc = gdppc_countries.set_index('gdppc')['Region'].to_dict()

x_vals, y_vals, sizes = pltkit.get_bubbles(counts_gdp, 1000)
x_vals_pc, y_vals_pc, sizes_pc = pltkit.get_bubbles(counts_gdppc, 1000)         
        

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))

ax1.scatter(x_vals, y_vals, s=sizes, alpha=0.8, color='steelblue', zorder=2)
labels_gdp = [mapping[val] if val in mapping else '' for val in values_gdp]
pltkit.stylize_axes(ax1, xscale='log', xlabel='GDP|PPP [billion US$2005]', xticks=values_gdp, 
                    xtickslabels=values_gdp.round(1), xtickslabels_kwargs={'rotation':90})

ax1_top = ax1.twiny()
pltkit.stylize_axes(ax1_top, xscale='log', xlim=ax1.get_xlim(), xticks=values_gdp, xlabel='Region', 
                    xtickslabels=labels_gdp, xtickslabels_kwargs={'rotation':90})


ax2.scatter(x_vals_pc, y_vals_pc, s=sizes_pc, alpha=0.8, color='darkolivegreen', zorder=2)
labels_gdppc = [mapping_gdppc[val] if val in mapping_gdppc else '' for val in values_gdppc]
pltkit.stylize_axes(ax2, xlabel='GDP|PPP per capita [billion US$2005]', xticks=values_gdppc, 
                    xtickslabels=values_gdppc.round(1), xtickslabels_kwargs={'rotation':90})


ax2_top = ax2.twiny()
pltkit.stylize_axes(ax2_top, xlim=ax2.get_xlim(), xticks=values_gdppc, xlabel='Region', 
                    xtickslabels=labels_gdppc, xtickslabels_kwargs={'rotation':90})

for ax in (ax1, ax2):
    pltkit.stylize_axes(ax, facecolor='whitesmoke', ylabel='Temperature Zone', grid=True,
                        grid_kwargs={'axis':'both', 'color':'white', 'zorder':0})

pltkit.stylize_plot(suptitle='Temperature Zone Frequencies for the original countries', tight_layout=True)

#### Population weighted 

In [ ]:
gdp_countries = gdp_data[(gdp_data['Region'].isin(pltkit.original_countries)) & (gdp_data['Scenario'] == 'SSP2')][['Region', 2020]].rename(columns={2020:'gdp'})
gdppc_countries = gdppc_data[(gdppc_data['Region'].isin(pltkit.original_countries)) & (gdppc_data['Scenario'] == 'SSP2')][['Region', 2020]].rename(columns={2020:'gdppc'})

counts_gdp, values_gdp, weights_gdp = get_values_counts_weights_for_bubbles(gdp_original.gdp, era5_tz.t2m, pop_ssp_year.GPOP, weighted=True)
counts_gdppc, values_gdppc, weights_gdppc = get_values_counts_weights_for_bubbles(gdp_original.gdppc, era5_tz.t2m, pop_ssp_year.GPOP, weighted=True)

mapping = gdp_countries.set_index('gdp')['Region'].to_dict()
mapping_gdppc = gdppc_countries.set_index('gdppc')['Region'].to_dict()

x_vals, y_vals, sizes = pltkit.get_bubbles_weighted_by_population(counts_gdp, weights_gdp, size_scale=1000)
x_vals_pc, y_vals_pc, sizes_pc = pltkit.get_bubbles_weighted_by_population(counts_gdppc, weights_gdppc, size_scale=1000)     

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))

ax1.scatter(x_vals, y_vals, s=sizes, alpha=0.8, color='steelblue', zorder=2)
labels_gdp = [mapping[val] if val in mapping else '' for val in values_gdp]
pltkit.stylize_axes(ax1, xscale='log', xlabel='GDP|PPP [billion US$2005]', xticks=values_gdp, 
                    xtickslabels=values_gdp.round(1), xtickslabels_kwargs={'rotation':90})

ax1_top = ax1.twiny()
pltkit.stylize_axes(ax1_top, xscale='log', xlim=ax1.get_xlim(), xticks=values_gdp, xlabel='Region', 
                    xtickslabels=labels_gdp, xtickslabels_kwargs={'rotation':90})


ax2.scatter(x_vals_pc, y_vals_pc, s=sizes_pc, alpha=0.8, color='darkolivegreen', zorder=2)
labels_gdppc = [mapping_gdppc[val] if val in mapping_gdppc else '' for val in values_gdppc]
pltkit.stylize_axes(ax2, xlabel='GDP|PPP per capita [billion US$2005]', xticks=values_gdppc, 
                    xtickslabels=values_gdppc.round(1), xtickslabels_kwargs={'rotation':90})


ax2_top = ax2.twiny()
pltkit.stylize_axes(ax2_top, xlim=ax2.get_xlim(), xticks=values_gdppc, xlabel='Region', 
                    xtickslabels=labels_gdppc, xtickslabels_kwargs={'rotation':90})

for ax in (ax1, ax2):
    pltkit.stylize_axes(ax, facecolor='whitesmoke', ylabel='Temperature Zone', grid=True,
                        grid_kwargs={'axis':'both', 'color':'white', 'zorder':0})

pltkit.stylize_plot(suptitle='Temperature Zone Frequencies for the original countries', tight_layout=True)

#### All countries

In [ ]:
gdp_2020 = gdp_data[gdp_data['Scenario'] == 'SSP2'][['Region', 2020]].rename(columns={2020:'gdp'})
gdppc_2020 = gdppc_data[gdppc_data['Scenario'] == 'SSP2'][['Region', 2020]].rename(columns={2020: 'gdppc'})

counts_gdp, values_gdp, _ = get_values_counts_weights_for_bubbles(gdp.gdp, era5_tz.t2m, pop_ssp_year.GPOP, weighted=False)
counts_gdppc, values_gdppc, _ = get_values_counts_weights_for_bubbles(gdp.gdppc, era5_tz.t2m, pop_ssp_year.GPOP, weighted=False)

mapping = gdp_2020.set_index('gdp')['Region'].to_dict()
mapping_gdppc = gdppc_2020.set_index('gdppc')['Region'].to_dict()

x_vals20, y_vals20, sizes20 = pltkit.get_bubbles(counts_gdp, 200)
x_vals_pc20, y_vals_pc20, sizes_pc20 = pltkit.get_bubbles(counts_gdppc, 200)


fig, (ax1, ax2) = plt.subplots(2,1, figsize=(30, 10))

ax1.scatter(x_vals20, y_vals20, s=sizes20, alpha=0.8, color='steelblue', zorder=2)
labels_gdp = [mapping[val] if val in mapping else '' for val in values_gdp]
pltkit.stylize_axes(ax1, xscale='log', xlabel='GDP|PPP [billion US$2005]', xticks=values_gdp, 
                    xtickslabels=values_gdp.round(1), xtickslabels_kwargs={'rotation':90})

ax1_top = ax1.twiny()
pltkit.stylize_axes(ax1_top, xscale='log', xlim=ax1.get_xlim(), xticks=values_gdp, xlabel='Region', 
                    xtickslabels=labels_gdp, xtickslabels_kwargs={'rotation':90})

ax2.scatter(x_vals_pc20, y_vals_pc20, s=sizes_pc20, alpha=0.8, color='darkolivegreen', zorder=2)
labels_gdppc = [mapping_gdppc[val] if val in mapping_gdppc else '' for val in values_gdppc]
pltkit.stylize_axes(ax2, xlabel='GDP|PPP per capita [billion US$2005]', xticks=values_gdppc, 
                    xtickslabels=values_gdppc.round(1), xtickslabels_kwargs={'rotation':90})

ax2_top = ax2.twiny()
pltkit.stylize_axes(ax2_top, xlim=ax2.get_xlim(), xticks=values_gdppc, xlabel='Region', 
                    xtickslabels=labels_gdppc, xtickslabels_kwargs={'rotation':90})

for ax in (ax1, ax2):
    pltkit.stylize_axes(ax, facecolor='whitesmoke', ylabel='Temperature Zone', grid=True,
                        grid_kwargs={'axis':'both', 'color':'white', 'zorder':0})

pltkit.stylize_plot(suptitle='Temperature Zone Frequencies for the original countries', tight_layout=True)

#### Population-weighted - Original countries

In [ ]:
gdp_2020 = gdp_data[gdp_data['Scenario'] == 'SSP2'][['Region', 2020]].rename(columns={2020:'gdp'})
gdppc_2020 = gdppc_data[gdppc_data['Scenario'] == 'SSP2'][['Region', 2020]].rename(columns={2020: 'gdppc'})

counts_gdp, values_gdp, weights_gdp = get_values_counts_weights_for_bubbles(gdp.gdp, era5_tz.t2m, pop_ssp_year.GPOP, weighted=True)
counts_gdppc, values_gdppc, weights_gdppc = get_values_counts_weights_for_bubbles(gdp.gdppc, era5_tz.t2m, pop_ssp_year.GPOP, weighted=True)

mapping = gdp_2020.set_index('gdp')['Region'].to_dict()
mapping_gdppc = gdppc_2020.set_index('gdppc')['Region'].to_dict()

x_vals20, y_vals20, sizes20 = pltkit.get_bubbles_weighted_by_population(counts_gdp, weights_gdp, size_scale=200)
x_vals_pc20, y_vals_pc20, sizes_pc20 = pltkit.get_bubbles_weighted_by_population(counts_gdppc, weights_gdppc, size_scale=200)


fig, (ax1, ax2) = plt.subplots(2,1, figsize=(30, 10))

ax1.scatter(x_vals20, y_vals20, s=sizes20, alpha=0.8, color='steelblue', zorder=2)
labels_gdp = [mapping[val] if val in mapping else '' for val in values_gdp]
pltkit.stylize_axes(ax1, xscale='log', xlabel='GDP|PPP [billion US$2005]', xticks=values_gdp, 
                    xtickslabels=values_gdp.round(1), xtickslabels_kwargs={'rotation':90})

ax1_top = ax1.twiny()
pltkit.stylize_axes(ax1_top, xscale='log', xlim=ax1.get_xlim(), xticks=values_gdp, xlabel='Region', 
                    xtickslabels=labels_gdp, xtickslabels_kwargs={'rotation':90})

ax2.scatter(x_vals_pc20, y_vals_pc20, s=sizes_pc20, alpha=0.8, color='darkolivegreen', zorder=2)
labels_gdppc = [mapping_gdppc[val] if val in mapping_gdppc else '' for val in values_gdppc]
pltkit.stylize_axes(ax2, xlabel='GDP|PPP per capita [billion US$2005]', xticks=values_gdppc, 
                    xtickslabels=values_gdppc.round(1), xtickslabels_kwargs={'rotation':90})

ax2_top = ax2.twiny()
pltkit.stylize_axes(ax2_top, xlim=ax2.get_xlim(), xticks=values_gdppc, xlabel='Region', 
                    xtickslabels=labels_gdppc, xtickslabels_kwargs={'rotation':90})

for ax in (ax1, ax2):
    pltkit.stylize_axes(ax, facecolor='whitesmoke', ylabel='Temperature Zone', grid=True,
                        grid_kwargs={'axis':'both', 'color':'white', 'zorder':0})

pltkit.stylize_plot(suptitle='Temperature Zone Frequencies for the original countries', tight_layout=True)

#### TMREL vs GDP

In [ ]:
tmrel = xr.open_dataset(f'{wdir}\\GBD_Data\\Exposure_Response_Functions\\TMRELs_2020.nc')

counts_gdp, values_gdp, weights_gdp = get_values_counts_weights_for_bubbles(gdp_original.gdp, tmrel, pop_ssp_year.GPOP, weighted=True)
counts_gdppc, values_gdppc, weights_gdppc = get_values_counts_weights_for_bubbles(gdp_original.gdppc, tmrel, pop_ssp_year.GPOP, weighted=True)

norm = Normalize(vmin=6, vmax=29) 
cmap = cm.get_cmap("copper")  

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6), sharey=True)

gdp_sorted = sorted(counts_gdp.keys())
gdppc_sorted = sorted(counts_gdppc.keys())

for i, gdp in enumerate(gdp_sorted):
    
    data = np.array(counts_gdp[gdp])
    weights = np.array(weights_gdp[gdp])

    mask = (~np.isnan(data)) & (~np.isnan(weights))
    data, weights = data[mask], weights[mask]

    if len(data) == 0:
        continue

    if len(data) > 50000:
        weights = weights / weights.sum()
        idx = np.random.choice(len(data), size=50000, replace=False, p=weights)
        data, weights = data[idx], weights[idx]

    weights = weights / weights.sum()

    kde = gaussian_kde(data, weights=weights)
    p5, p95 = np.percentile(data, [5, 95])
    x = np.linspace(p5, p95, 200)
    y = kde(x)
    scale = 0.03 / y.max()
    y_scaled = y * scale

    for xi, yi in zip(x[:-1], y_scaled[:-1]):
        ax1.fill_betweenx([xi, xi + 0.1], gdp, 10**(np.log10(gdp) + yi),
                      color=cmap(norm(xi)), alpha=0.8, zorder=2)

labels_gdp = [mapping[val] if val in mapping else '' for val in gdp_sorted]

pltkit.stylize_axes(ax1, xscale='log', xlabel='GDP|PPP [billion US$2005]', xticks=gdp_sorted, 
                    xtickslabels=np.round(gdp_sorted, 1), xtickslabels_kwargs={'rotation':90})

ax1_top = ax1.twiny()
pltkit.stylize_axes(ax1_top, xscale='log', xlim=ax1.get_xlim(), xticks=gdp_sorted, xlabel='Region', 
                    xtickslabels=labels_gdp, xtickslabels_kwargs={'rotation':90})


# ---------- GDP per Capita Plot ----------
for i, gdppc in enumerate(gdppc_sorted):
    data = np.array(counts_gdppc[gdppc])
    weights = np.array(weights_gdppc[gdppc])

    mask = (~np.isnan(data)) & (~np.isnan(weights))
    data, weights = data[mask], weights[mask]

    if len(data) == 0:
        continue

    if len(data) > 50000:
        weights = weights / weights.sum()
        idx = np.random.choice(len(data), size=50000, replace=False, p=weights)
        data, weights = data[idx], weights[idx]

    weights = weights / weights.sum()

    kde = gaussian_kde(data, weights=weights)
    p5, p95 = np.percentile(data, [5, 95])
    x = np.linspace(p5, p95, 200)
    y = kde(x)
    scale = 0.6 / y.max()
    y_scaled = y * scale

    for xi, yi in zip(x, y_scaled):
        ax2.fill_betweenx([xi, xi + 0.2], gdppc, gdppc + yi,
                      color=cmap(norm(xi)), alpha=0.8, zorder=2)

labels_gdppc = [mapping_gdppc[val] if val in mapping_gdppc else '' for val in gdppc_sorted]
pltkit.stylize_axes(ax2, xlabel='GDP|PPP per capita [billion US$2005]', xticks=gdppc_sorted, 
                    xtickslabels=np.round(gdppc_sorted, 1), xtickslabels_kwargs={'rotation':90})

ax2_top = ax2.twiny()
pltkit.stylize_axes(ax2_top, xscale='log', xlim=ax2.get_xlim(), xticks=gdppc_sorted, xlabel='Region', 
                    xtickslabels=labels_gdppc, xtickslabels_kwargs={'rotation':90})

for ax in (ax1, ax2):
    pltkit.stylize_axes(ax, facecolor='whitesmoke', ylabel='TMREL', grid=True,
                        grid_kwargs={'axis':'both', 'color':'white', 'zorder':0})

pltkit.stylize_plot(suptitle='Population-weighted Distribution of Temperature Zones by GDP and GDP per Capita', tight_layout=True)

#### All countries

In [ ]:
gdp = xr.open_dataset(f'{wdir}\\SocioeconomicData\\EconomicData\\GDP_SSP2_2020.nc')
gdp_2020 = gdp_data[gdp_data['Scenario'] == 'SSP2'][['Region', 2020]].rename(columns={2020:'gdp'})
gdppc_2020 = gdppc_data[gdppc_data['Scenario'] == 'SSP2'][['Region', 2020]].rename(columns={2020: 'gdppc'})

counts_gdp_20, values_gdp_20, weights_gdp_20 = pltkit.get_values_counts_weights_for_bubbles(gdp.gdp, tmrel, pop_ssp_year.GPOP, weighted=True)
counts_gdppc_20, values_gdppc_20, weights_gdppc_20 = pltkit.get_values_counts_weights_for_bubbles(gdp.gdppc, tmrel, pop_ssp_year.GPOP, weighted=True)

mapping = gdp_2020.set_index('gdp')['Region'].to_dict()
mapping_gdppc = gdppc_2020.set_index('gdppc')['Region'].to_dict()

x_vals20, y_vals20, sizes20 = pltkit.get_bubbles_weighted_by_population(counts_gdp_20, weights_gdp_20, size_scale=200)
x_vals_pc20, y_vals_pc20, sizes_pc20 = pltkit.get_bubbles_weighted_by_population(counts_gdppc_20, weights_gdppc_20, size_scale=200)

fig, (ax1, ax2) = plt.subplots(2,1, figsize=(30, 10))

ax1.scatter(x_vals20, y_vals20, s=sizes20, alpha=0.8, color='steelblue', zorder=2)
labels_gdp_20 = [mapping[val] if val in mapping else '' for val in values_gdp_20]
pltkit.stylize_axes(ax1, xscale='log', xlabel='GDP|PPP [billion US$2005]', xticks=values_gdp_20, xtickslabels=values_gdp_20.round(1), xtickslabels_kwargs={'rotation':90})
ax1_top = ax1.twiny()
pltkit.stylize_axes(ax1_top, xscale='log', xlim=ax1.get_xlim(), xticks=values_gdp_20, xtickslabels=labels_gdp_20, xtickslabels_kwargs={'rotation':90})

ax2.scatter(x_vals_pc20, y_vals_pc20, s=sizes_pc20, alpha=0.8, color='darkolivegreen', zorder=2)
labels_gdppc_20 = [mapping[val] if val in mapping else '' for val in values_gdppc_20]
pltkit.stylize_axes(ax2, xlabel='GDP|PPP per capita [billion US$2005]', xticks=values_gdppc_20, xtickslabels=values_gdppc_20.round(1), xtickslabels_kwargs={'rotation':90})
ax2_top = ax2.twiny()
pltkit.stylize_axes(ax2_top, xlim=ax2.get_xlim(), xticks=values_gdppc_20, xtickslabels=labels_gdppc_20, xtickslabels_kwargs={'rotation':90})

for ax in (ax1, ax2):
    pltkit.stylize_axes(ax, facecolor='whitesmoke', ylabel='TMREL', grid=True, grid_kwargs={'axis':'both', 'color':'white', 'zorder':0})
    
pltkit.stylize_plot(suptitle='TMREL frequencies per country', tight_layout=True)

### ERF

#### Plot one disease

In [ ]:
disease = 'ckd'; tz = 6
erf = pd.read_csv(f'{wdir}\\data\\Response_Functions\\erf_reformatted2\\erf_{disease}.csv', index_col=[1,2])

plt.plot(erf.loc[tz].index, erf.loc[tz, 'rr'])
pltkit.stylize_plot(grid=True, xlabel='Daily temperature (°C)', ylabel='RR', title=f'RR for {disease} - {tz}℃')

In [ ]:
tz = 6
start = 10
end = 40

xnew_hot, ynew_hot, xnew_cold, ynew_cold = pltkit.interpolate_erf(start_graph=start, end_graph=end, erf=erf, tz=tz)

plt.plot(erf.loc[tz].index, erf.loc[tz, 'rr'])
plt.plot(xnew_hot, ynew_hot, 'C0--')
plt.plot(xnew_cold, ynew_cold, 'C0--')
pltkit.stylize_plot(xlabel='Daily temperature (°C)', ylabel='RR', title=f'ERF - {disease} - {tz}℃', grid=True, show=True)

#### Meta-ERFs

In [76]:
erf_dic ={}
for disease in pltkit.diseases.keys():
    erf_dic[disease] = pd.read_csv(f'{wdir}\\data\\Response_Functions\\erf_reformatted2\\erf_{disease}.csv', index_col=[1,2])

In [ ]:
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.magma(np.linspace(0,1,23)))

fig, axs = plt.subplots(2,4, figsize=(18,8))#, dpi=300)
axs = axs.flatten()

for i, disease in enumerate(pltkit.relevant_diseases.keys()):
    for tz in range(6,29):
        axs[i].plot(erf_dic[disease].loc[tz].index, erf_dic[disease].loc[tz, 'rr'], label = tz)

    pltkit.stylize_axes(axs[i], facecolor='whitesmoke', grid=True, grid_kwargs={'c':'white'}, title=f'{pltkit.relevant_diseases[disease]}', ylim=(0.9,2))
    for i in [0,4]:
        pltkit.stylize_axes(axs[i], ylabel='Relative Risk')
    for i in range(4,8):
        pltkit.stylize_axes(axs[i], xlabel='Daily temperature [°C]')
    
plt.legend(loc='upper center', bbox_to_anchor=(-1.3, -0.2), ncol=8, title = 'Temperature zones')

#### Shifting horizontally ERF

In [ ]:
fig, axs = plt.subplots(2,4, figsize=(18,8))#, dpi=300)
axs = axs.flatten()

for i, disease in enumerate(pltkit.relevant_diseases.keys()):
    for tz in range(6,29):
        x = erf_dic[disease].loc[tz].index.values        # valores de temperatura
        y = erf_dic[disease].loc[tz, 'rr'].values        # riesgo relativo

        x_shift = x - x[np.argmin(y)]                    # trasladar mínimo a 0
        axs[i].plot(x_shift, y, label=tz)

    pltkit.stylize_axes(axs[i], facecolor='whitesmoke', grid=True, grid_kwargs={'c':'white'}, title=f'{pltkit.relevant_diseases[disease]}', ylim=(0.9,2))
    for i in [0,4]:
        pltkit.stylize_axes(axs[i], ylabel='Relative Risk')
    for i in range(4,8):
        pltkit.stylize_axes(axs[i], xlabel='$\Delta$ Daily temperature [°C]')
    
plt.legend(loc='upper center', bbox_to_anchor=(-1.3, -0.2), ncol=8, title = 'Temperature zones')

#### Rough examination of the mean TMREL vs TZ

In [ ]:
mpl.rcParams["axes.prop_cycle"] = mpl.rcParamsDefault["axes.prop_cycle"]
fig, ax = plt.subplots(1, 1)  # , dpi=300

for i, disease in enumerate(pltkit.relevant_diseases.keys()):
    tzs = []
    tmrels = []
    for tz in range(6, 29):
        rr_min = np.argmin(erf_dic[disease].loc[tz, 'rr'].values)
        tmrel = erf_dic[disease].loc[tz].index.values[rr_min]
        tzs.append(tz)
        tmrels.append(tmrel)
    
    ax.scatter(tzs, tmrels, c=[pltkit.original_colors[i]], s=5, label=pltkit.relevant_diseases[disease])

ax.legend(fontsize=8)
ax.grid()

#### Extrapolating ERF

In [ ]:
start_graph = -20
end_graph = 50

interp_hot_dic = defaultdict(dict); interp_cold_dic = defaultdict(dict)
xnew_hot_dic = defaultdict(dict); ynew_hot_dic = defaultdict(dict)
xnew_cold_dic = defaultdict(dict); ynew_cold_dic = defaultdict(dict)

for disease in pltkit.relevant_diseases.keys():
    for tz in range(6,29):        
        xnew_hot_dic[disease][tz], ynew_hot_dic[disease][tz], xnew_cold_dic[disease][tz], ynew_cold_dic[disease][tz] = pltkit.interpolate_erf(start_graph, end_graph, erf_dic[disease], tz)


tz_values = list(range(6, 29))  # 6 a 28 inclusive
colors = plt.cm.magma(np.linspace(0, 1, len(tz_values)))
tz_color_map = {tz: color for tz, color in zip(tz_values, colors)}

fig, axs = plt.subplots(2,4, figsize=(18,8))#, dpi=300)
axs = axs.flatten()

for i, disease in enumerate(pltkit.relevant_diseases.keys()):
    for tz in range(6,29):
        color = tz_color_map[tz]
        axs[i].plot(erf_dic[disease].loc[tz].index, erf_dic[disease].loc[tz, 'rr'], label = tz, color=color)
        axs[i].plot(xnew_hot_dic[disease][tz], ynew_hot_dic[disease][tz], '--', color=color)
        axs[i].plot(xnew_cold_dic[disease][tz], ynew_cold_dic[disease][tz], '--', color=color)
        
    pltkit.stylize_axes(axs[i], facecolor='whitesmoke', grid=True, grid_kwargs={'c':'white'}, title=f'{pltkit.relevant_diseases[disease]}', ylim=(0.9,5))
    for i in [0,4]:
        axs[i].set_ylabel('Relative Risk')
    for i in range(4,8):
        axs[i].set_xlabel('Daily temperature [°C]')
 
plt.legend(loc='upper center', bbox_to_anchor=(-1.3, -0.2), ncol=8, title = 'Temperature zones')

### Initial plots 

In [ ]:
### AR6 pathways
ar6 = pd.read_csv('X:\\user\\dekkerm\\Data\\AR6_snapshots\\AR6_snapshot_Nayeli_global.csv')
cc_path_mean = ar6.iloc[:, 10:-1].groupby(ar6['Category']).mean()
cc_path_std = ar6.iloc[:, 10:-1].groupby(ar6['Category']).std()

fig, ax = plt.subplots(figsize=(8, 5))
for i,categories in enumerate(ar6['Category'].unique()):
    plt.plot(cc_path_mean.columns, cc_path_mean.loc[categories], label=categories, color=pltkit.original_colors[i])
    plt.fill_between(cc_path_mean.columns, cc_path_mean.loc[categories] - 2*cc_path_std.loc[categories], cc_path_mean.loc[categories] + 2*cc_path_std.loc[categories], 
                     color=pltkit.original_colors[i], alpha=0.2)
ax.set_xticks(cc_path_mean.columns[::10])
pltkit.stylize_plot(title='CC pathways - AR6 \n(2std uncertainty range)', ylabel='GMST anomaly (°C)', xlabel='Year', legend=True, show=True)

In [ ]:
# Temperature zone
era5_tz = xr.open_dataset('X:\\user\\liprandicn\\Health Impacts Model\\ClimateData\\ERA5\\ERA5_mean_1980-2019_land_t2m_tz.nc')
fig = plt.figure(figsize=(10,5), dpi=300)
ax = fig.add_subplot(111, projection=ccrs.PlateCarree(central_longitude=0), frameon=False)
ax.coastlines(resolution='10m', lw = 0.1)
era5_tz.t2m.plot(ax=ax, transform=ccrs.PlateCarree(), cbar_kwargs = {'orientation': 'horizontal', 'shrink':0.6, 'pad':0.05, 'aspect':20, 'label':'Temperature zones'},
                 vmin=6, vmax=28, cmap = 'cividis')

In [ ]:
# Example of daily temperature generation
year = 2020
std = 5
ccategory = 'C1'
noise, noise_leap = create_noise(std)
gcm_diff, gcm_start, gcm_end, cc_path_mean = read_climate_data()
daily_temp, num_days = daily_temp_interp('C1', year, cc_path_mean, gcm_diff, gcm_start, gcm_end, noise, noise_leap)
daily_temp = xr.DataArray(daily_temp, dims=["lat", "lon", 'time'], 
                          coords={"lat": era5_tz.latitude.values, 'lon': era5_tz.longitude.values, 'time': pd.date_range(start=f'01/01/{year}', end=f'31/12/{year}')})

fig = plt.figure(figsize=(10,5), dpi=300)
ax = fig.add_subplot(111, projection=ccrs.PlateCarree(central_longitude=0), frameon=False)
ax.coastlines(resolution='10m', lw = 0.1)
daily_temp.isel(time=0).plot(ax=ax, cbar_kwargs = {'orientation': 'horizontal', 'shrink':0.6, 'pad':0.05, 'aspect':20, 'label':'Daily temperature [°C]'},
                 cmap = 'coolwarm', vmin=-30, vmax=50) 

In [ ]:
fig = plt.figure(figsize=(10,5), dpi=300)
ax = fig.add_subplot(111, projection=ccrs.PlateCarree(central_longitude=0), frameon=False)
ax.coastlines(resolution='10m', lw = 0.1)
daily_temp.isel(time=0).plot(ax=ax, cbar_kwargs = {'orientation': 'horizontal', 'shrink':0.6, 'pad':0.05, 'aspect':20, 'label':'Daily temperature [°C]'},
                 cmap = 'coolwarm', vmin=-30, vmax=50) 

In [ ]:
# Example of daily RR 
disease = 'resp_copd'
day = 1
erf = pd.read_csv(f'{wdir}\\ResponseFunctions\\erf_reformatted\\erf_{disease}_mean.csv', header=None).to_numpy()
day_rr = get_daily_rr(era5_tz.t2m.values, daily_temp.values[:,:,day], erf)
day_rr = xr.DataArray(day_rr, dims=["lat", "lon"], 
                          coords={"lat": era5_tz.latitude.values, 'lon': era5_tz.longitude.values})
day_rr.plot(cmap = 'Purples', vmax=1.4)  

### Plots comparing methods

In [3]:
# Open excel created with data from Zhao et al. 2021
zhao_df = pd.read_excel(f'{wdir}\\analysis\\zhao2021_tab1.xlsx', header=[0,1], index_col=0)
regions_class = pd.read_excel(f'{wdir}\\data\\IMAGE_regions\\regions_comparisson.xlsx')

In [4]:
# Load GBD mortality data
gbd_df = pd.read_csv(f'{wdir}\\data\\GBD_data\\Mortality\\IHME-GBD_2021_DATA.csv')

In [11]:
# Load approach 1 results

gbd_ = gbd_df[
    (gbd_df['cause_name'].isin(pltkit.diseases.values())) &
    (gbd_df['sex_name'] == 'Both') &
    (gbd_df['year'].isin(range(2000, 2020))) &
    (gbd_df['age_name'] == 'All ages') &
    (gbd_df['location_name'] != 'Global')][['location_id', 'location_name', 'cause_name', 'year', 'val', 'upper', 'lower']]	
gbd_['year']=gbd_['year'].astype(int)

app1 = pd.read_csv(f'{wdir}\\output\\paf_era5_GBD_level3_all-dis_1erf_2000-2019.csv', header=[0,1,2], index_col=[0])
app2 = pd.read_csv(f'{wdir}\\output\\paf_era5_GBD_level3_all-dis_2000-2019.csv', header=[0,1,2], index_col=[0])
app3 = pd.read_csv(f'{wdir}\\output\\paf_era5_GBD_level3_all-dis_extrap_2000-2019.csv', header=[0,1,2], index_col=[0])
app4 = pd.read_csv(f'{wdir}\\output\\paf_era5_GBD_level3_all-dis_1erf_extrap_2000-2019.csv', header=[0,1,2], index_col=[0])

def process_app(app):
    
    # Select deaths from both heat and cold
    app = app.xs('all', level=2, axis=1)
    # Stack dataframe to have location, year, cause_key as index and paf as column
    app = app.stack(level=[0,1], future_stack=True).reset_index().rename(columns={'level_0':'location_id', 'level_1':'year', 'level_2':'cause_key', 0:'paf'})
    # Map diseases names with keys
    app['cause_name'] = app['cause_key'].map({k:v for k,v in pltkit.diseases.items()})
    app['year'] = app['year'].astype(int)
    # Merge with mortality data
    app = app.merge(gbd_, on=['location_id', 'cause_name', 'year'], how='left')
    # Calculate deaths attributable to temperature
    app['mean'] = app['val'] * app['paf'] 
    app['upper_est'] = app['upper'] * app['paf'] 
    app['lower_est'] = app['lower'] * app['paf'] 
    app = app[['year', 'cause_name', 'location_name', 'mean', 'upper_est', 'lower_est']]  
    # Merge with regions classification
    app = app.merge(regions_class[['gbd_level3_mor', 'continents']], left_on='location_name', right_on='gbd_level3_mor', how='left')
    app = app.groupby(['year','continents']).sum().reset_index()[['continents', 'year', 'mean', 'upper_est', 'lower_est']]
    app = app.groupby(['continents']).mean().reset_index()
    
    return app

app1 = process_app(app1)
app2 = process_app(app2)
app3 = process_app(app3)
app4 = process_app(app4)

In [ ]:
import plotly.graph_objects as go

df0 = zhao_df.xs('overall', axis=1, level=0).iloc[1:,:]
df1 = app1.set_index('continents').loc[df0.index]
df2 = app2.set_index('continents').loc[df0.index]
df3 = app3.set_index('continents').loc[df0.index]
df4 = app4.set_index('continents').loc[df0.index]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df0.index,
    y=df0["mean"],
    mode="markers",
    name = 'Zhao et al. 2021',
    error_y=dict(
        type="data",
        symmetric=False,
        array=df0["p95"] - df0["mean"],
        arrayminus=df0["mean"] - df0["p5"]
    ),
    marker=dict(size=10, color="cadetblue")
))

dfs = [df1, df2, df3]#, df4]
names = ['GBD (1ERF)', 'GBD', 'GBD (ext)']#, 'GBD (1ERF & ext)']
colors = ['lightcoral', 'sienna', 'rosybrown']#, 'darkorange']

for df, name, color in zip(dfs, names, colors):

    fig.add_trace(go.Scatter(
        x=df.index,
        y=df["mean"],
        mode="markers",
        name = name,
        error_y=dict(
            type="data",
            symmetric=False,
            array=df["upper_est"] - df["mean"],
            arrayminus=df["mean"] - df["lower_est"]
        ),
        marker=dict(size=10, color=color)
    ))


fig.update_layout(
    title="Estimation including 5–95% range",
    yaxis_title="Average annual deaths (2000-2019)",
    xaxis_title="Continent",
    height=500,
    width=700,
)

fig.show()